In [16]:
from flask import Response

In [ ]:
class myException(Ex)

Custom Exception Defintions
- NotCoreObject √
- UnsupportedGuid √
- InvalidParent
- EmptyAuth
- NoId


In [33]:
class NotCoreObject(Exception):
    """Error: Object isn't a core object
    
    """
    def __init__(self, payload):
        self.message = {"obj": payload,"message": "Object keys don't match any core metadata object"} 
        
    def output(self):
        return Response(
            status = 400,
            response = json.dumps(self.message),
                        mimetype = 'application/json')
    
    

In [30]:
class UnsupportedGuid(Exception):
    """ Error: Id isn't an ARK or DOI
    
    """
    def __init__(self, Guid):
        self.message = {"Guid": Guid, "message": "Guid is neither an Ark nor a DOI"}
    
    def output(self):
        return Response(
            status = 404,
            response = json.dumps(self.message),
            mimetype = 'application/json'
        )

In [34]:
class InvalidParent(Exception):
    """ Error: Object has no parent object in the neo cache
    """
    def __init__(self, Parent):
        self.message = {"Parent": Parent, "message": "No record of parent in the cache"}
        
    def output(self):
        return Response(
            status = 404,
            response = json.dumps(self.message),
            mimetype = 'application/json'
        )

In [32]:
example_string = "hi hi"
try:
    raise NotCoreObject(obj= example_string)
except NotCoreObject as err:
    print(err.output())

<Response likely-streamed [400 BAD REQUEST]>


In [26]:
request = {}

In [27]:
import requests

In [29]:
basic_auth = requests.auth.HTTPBasicAuth(None, None)

In [35]:
from neo4j.v1 import GraphDatabase

In [36]:
neo_driver_fail = GraphDatabase.driver("bolt://localhost:9090")

ServiceUnavailable: Cannot acquire connection to Address(host='localhost', port=9090)

Using boto3 for testing
    - grab the public ip's for the tests

In [1]:
import boto3

In [2]:
client = boto3.client('ecs')

In [4]:
client.list_clusters()

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '126',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 05 Jun 2018 22:08:15 GMT',
   'server': 'Server',
   'x-amzn-requestid': 'f1b5619d-690c-11e8-9b0a-9df36ce44c46'},
  'HTTPStatusCode': 200,
  'RequestId': 'f1b5619d-690c-11e8-9b0a-9df36ce44c46',
  'RetryAttempts': 0},
 'clusterArns': ['arn:aws:ecs:us-east-1:280922329489:cluster/ORS_Test',
  'arn:aws:ecs:us-east-1:280922329489:cluster/mycluster']}

In [59]:
CLUSTER = 'arn:aws:ecs:us-east-1:280922329489:cluster/ORS_Test'

In [81]:
taskMetadata = client.list_tasks(
    cluster = 'arn:aws:ecs:us-east-1:280922329489:cluster/ORS_Test',
    serviceName = 'wsgi_service'
)

In [82]:
taskArn = taskMetadata['taskArns'][0]

In [83]:
taskArn

'arn:aws:ecs:us-east-1:280922329489:task/35a0b838-afac-4f60-a9d9-8a5410c8548c'

In [62]:
tasks = client.describe_tasks(
    cluster = CLUSTER,
    tasks = [taskArn]
)

In [63]:
 network_interface = [element['value'] for element in tasks['tasks'][0]['attachments'][0]['details'] if element['name'] == "networkInterfaceId"]

In [64]:
#  aws ec2 describe-network-interfaces --network-interface-ids eni-xxxxxxxx
ec2 = boto3.resource('ec2')
network_interface = ec2.NetworkInterface(network_interface[0])

In [66]:
#network_interface.describe_attribute('public_ip_address')
network_interface.load()

In [77]:
network_interface.association_attribute['PublicIp']

'54.144.27.239'

In [16]:
for element in tasks['tasks']['attachments']:
    if element['name'] == ''

[{'id': 'f57ca60e-000f-4fe1-89c3-7688dd654bff', 'type': 'ElasticNetworkInterface', 'status': 'ATTACHED', 'details': [{'name': 'subnetId', 'value': 'subnet-47d61d1b'}, {'name': 'networkInterfaceId', 'value': 'eni-e6cd3b7c'}, {'name': 'macAddress', 'value': '0e:62:a9:3f:98:70'}, {'name': 'privateIPv4Address', 'value': '10.0.1.246'}]}]


In [88]:
# grab cloudwatch logs
cloudwatch = boto3.client('logs')

In [89]:
# grab log groups
cloudwatch.list_dashboards()

{'DashboardEntries': [],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '289',
   'content-type': 'text/xml',
   'date': 'Wed, 06 Jun 2018 22:48:52 GMT',
   'x-amzn-requestid': 'c9078f02-69db-11e8-8879-e372925e7cec'},
  'HTTPStatusCode': 200,
  'RequestId': 'c9078f02-69db-11e8-8879-e372925e7cec',
  'RetryAttempts': 0}}

In [34]:
import re

In [35]:
identifier = "ark:/99999/fk4r8776t"

In [37]:
if re.match("ark:/", identifier):
    print("hi")

hi


In [38]:
bytes_example = b'abcd'

In [39]:
bytes_example.decode('utf-8')

'abcd'

In [90]:
test_response = b'success: ark:/99999/fk4r8776t\nminid.checksumMethod: sha1\n_profile: minid\nminid.identifier: ark:/99999/fk4r8776t\nminid.titles: minid: A BD2K Minimal Viable Identifier Pilot v0.1\nminid.status: ACTIVE\n_target: https://ezid.cdlib.org/id/ark:/99999/fk4r8776t\n_ownergroup: apitest\n_status: reserved\n_export: yes\nminid.checksum: cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f\nminid.locations: http://bd2k.ini.usc.edu/assets/all-hands-meeting/minid_v0.1_Nov_2015.pdf\n_updated: 1528331650\nminid.creator: 0000-0003-2129-5269\nminid.created: 2015-11-10 04:44:44.387671\n_owner: apitest\n_created: 1528331650\n'

In [93]:
def digestANVL(response_text):
    ANVLdict = {}
    if isinstance(response_text, bytes):
        tempANVL = response_text.decode('UTF-8')
    else:
        tempANVL = response_text
        
    for element in tempANVL.split("\n"):
        SplitUp = element.split(": ", 1)
        if len(SplitUp)>1:
            ANVLdict[SplitUp[0]] = SplitUp[1]
    return ANVLdict

In [94]:
digestANVL(test_response)

{'_created': '1528331650',
 '_export': 'yes',
 '_owner': 'apitest',
 '_ownergroup': 'apitest',
 '_profile': 'minid',
 '_status': 'reserved',
 '_target': 'https://ezid.cdlib.org/id/ark:/99999/fk4r8776t',
 '_updated': '1528331650',
 'minid.checksum': 'cacc1abf711425d3c554277a5989df269cefaa906d27f1aaa72205d30224ed5f',
 'minid.checksumMethod': 'sha1',
 'minid.created': '2015-11-10 04:44:44.387671',
 'minid.creator': '0000-0003-2129-5269',
 'minid.identifier': 'ark:/99999/fk4r8776t',
 'minid.locations': 'http://bd2k.ini.usc.edu/assets/all-hands-meeting/minid_v0.1_Nov_2015.pdf',
 'minid.status': 'ACTIVE',
 'minid.titles': 'minid: A BD2K Minimal Viable Identifier Pilot v0.1',
 'success': 'ark:/99999/fk4r8776t'}